In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import requests, re, time
from bs4 import BeautifulSoup
import scraping_class
import pprint
import pandas as pd
logfile = 'log.csv'## name your log file.
connector = scraping_class.Connector(logfile)
session = requests.session()
session.headers['wmd776@alumni.ku.dk'] = 'wmd776@alumni.ku.dk' 
session.headers['Amalie Tokkesdal'] = 'Amalie Tokkesdal'
session.headers

{'User-Agent': 'python-requests/2.21.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'wmd776@alumni.ku.dk': 'wmd776@alumni.ku.dk', 'Amalie Tokkesdal': 'Amalie Tokkesdal'}

# Importing the packages

# Importing and parsing the data

In [2]:
url='https://www.imdb.com/chart/top'
number_of_movies = 250
response = requests.get(url)

if response.ok:
    html = response.text
else:
    print('error')
    
soup = BeautifulSoup(html,'html.parser')

link_location = html.split('href="')[1:]

links = set()


for link_loc in link_location:
    link = link_loc.split('"')[0]
    if '/title/' in link:
        links.add(link) 
links = ['https://www.imdb.com/'+link for link in links if not 'ref' in link]# add the domain to each link

links

['https://www.imdb.com//title/tt0081505/',
 'https://www.imdb.com//title/tt0116282/',
 'https://www.imdb.com//title/tt0071562/',
 'https://www.imdb.com//title/tt0107207/',
 'https://www.imdb.com//title/tt0060196/',
 'https://www.imdb.com//title/tt0107290/',
 'https://www.imdb.com//title/tt0070735/',
 'https://www.imdb.com//title/tt0253474/',
 'https://www.imdb.com//title/tt3011894/',
 'https://www.imdb.com//title/tt0072684/',
 'https://www.imdb.com//title/tt0053198/',
 'https://www.imdb.com//title/tt0092005/',
 'https://www.imdb.com//title/tt1954470/',
 'https://www.imdb.com//title/tt0169102/',
 'https://www.imdb.com//title/tt1280558/',
 'https://www.imdb.com//title/tt0169547/',
 'https://www.imdb.com//title/tt0097165/',
 'https://www.imdb.com//title/tt0071853/',
 'https://www.imdb.com//title/tt0361748/',
 'https://www.imdb.com//title/tt2582802/',
 'https://www.imdb.com//title/tt0086879/',
 'https://www.imdb.com//title/tt0119217/',
 'https://www.imdb.com//title/tt1255953/',
 'https://w

In [3]:
def find_budget(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Budget:":
            return float(i.text.strip().split("\n")[0].split("$")[1].replace(",", ""))
        
def find_gross(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Cumulative Worldwide Gross:":
            return float(i.text.strip().split("\n")[0].split("$")[1].replace(",", ""))
        
def find_grossUSA(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Gross USA:":
            return float(i.text.strip().split("\n")[0].split("$")[1].replace(",", ""))

def find_rating(soup):
    for i in soup.find(id="title-overview-widget").find_all("div", class_="ratingValue"):
        return i.text.strip().split("/")[0]
        
def find_runtime(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Runtime:":
            return float(i.text.strip().split("\n")[1].split("min")[0].strip())
def find_date(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Release Date:":
            return i.text.strip().split(":")[1].split("(")[0].strip()
        
def find_genre(soup):
    for i in soup.find(id="title-overview-widget").find_all("div", class_="subtext"):
        if str(i.text.strip().split("|")[2].replace('\n','').replace(' ','').split(","))[-3]==(')'):
            return i.text.strip().split("|")[1].replace('\n','').replace(' ','').split(",")
        else:
            return i.text.strip().split("|")[2].replace('\n','').replace(' ','').split(",")
        
def preparedatetime(dataframe):
    dataframe['datetime'] = pd.to_datetime(dataframe['date'], format = '%d %B %Y')
    dataframe['month'] = dataframe.datetime.dt.month
    dataframe['month'] = dataframe.datetime.dt.day
    dataframe['month'] = dataframe.datetime.dt.year

In [4]:

def page_extractor(url):
    r, call_id = connector.get(url, "LOL")
    try:
        soup = BeautifulSoup(r.text, "html.parser")
        title, year = soup.select("#title-overview-widget > div.vital > div.title_block > div > div.titleBar > div.title_wrapper > h1")[0].text.split("\xa0")
        budget = find_budget(soup)
        cumulative_worldwide_gross=find_gross(soup)
        gross_usa=find_grossUSA(soup)
        rating = find_rating(soup)
        runtime = find_runtime(soup)
        date= find_date(soup)
        genre = find_genre(soup)
    except:
        soup = None
        title = None
        budget = None
        cumulative_worldwide_gross= None
        gross_usa = None
        rating=None
        runtime = None
        date = None
        genre = None
        print(url, "didnt work")
    return {
        "title": title,
        "year": year,
        "budget": budget,
        "url": url,
        "cumulative_world_gross": cumulative_worldwide_gross,
        "gross_usa": gross_usa,
        "rating": rating,
        "runtime": runtime,
        "date": date,
        "genre": genre
    }

#Husk at finde titel på engelsk

### Create a list and append information from every movie on the list. 
Remark: We have only chosen the first 15 so far, as to not make the coding too slow in progress. If you wish to see the whole list simply delete the [:15]


In [5]:
res = []
for idx, i in enumerate(links[:15]):
    print(idx + 1, "of", len(links[:15]))
    res.append(page_extractor(i))

1 of 15
2 of 15
3 of 15
4 of 15
5 of 15
6 of 15
7 of 15
8 of 15
9 of 15
10 of 15
11 of 15
12 of 15
13 of 15
https://www.imdb.com//title/tt1954470/ didnt work
14 of 15
15 of 15


### Making our list into a dataframe. 

A dataframe is basically just when a list has both several columns and rows, if there's only one row it's an array.

In [10]:
df = pd.DataFrame(res)


df['datetime'] = pd.to_datetime(df['date'], format = '%d %B %Y')
df

,budget,cumulative_world_gross,date,genre,gross_usa,rating,runtime,title,url,year,datetime
0,19000000.0,NaN,26 September 1980,"[Drama, Horror]",44017374.0,8.4,144.0,Ondskabens hotel,https://www.imdb.com//title/tt0081505/,(1980),1980-09-26
1,7000000.0,3.600000e+07,9 August 1996,"[Crime, Drama, Thriller]",24611975.0,8.1,98.0,Fargo,https://www.imdb.com//title/tt0116282/,(1996),1996-08-09
2,13000000.0,NaN,29 August 1975,"[Crime, Drama]",57300000.0,9.0,202.0,Godfather del 2,https://www.imdb.com//title/tt0071562/,(1974),1975-08-29
3,13000000.0,6.579686e+07,18 March 1994,"[Biography, Drama]",25010410.0,8.1,133.0,In the Name of the Father,https://www.imdb.com//title/tt0107207/,(1993),1994-03-18
4,1200000.0,NaN,8 April 1968,[Western],6100000.0,8.8,161.0,"Den gode, den onde og den grusomme",https://www.imdb.com//title/tt0060196/,(1966),1968-04-08
5,63000000.0,1.029154e+09,17 September 1993,"[Action, Adventure, Sci-Fi]",402453882.0,8.1,127.0,Jurassic Park,https://www.imdb.com//title/tt0107290/,(1993),1993-09-17
6,5500000.0,NaN,15 April 1974,"[Comedy, Crime, Drama]",159600000.0,8.3,129.0,Sidste stik,https://www.imdb.com//title/tt0070735/,(1973),1974-04-15
7,35000000.0,1.200726e+08,4 July 2003,"[Biography, Drama, Music]",32572577.0,8.5,150.0,Pianisten,https://www.imdb.com//title/tt0253474/,(2002),2003-07-04
8,3300000.0,3.635375e+07,7 May 2015,"[Comedy, Drama, Thriller]",3107072.0,8.1,122.0,Wild Tales,https://www.imdb.com//title/tt3011894/,(2014),2015-05-07
9,11000000.0,3.150000e+07,10 September 1976,"[Adventure, Drama, History]",20000000.0,8.1,185.0,Barry Lyndon,https://www.imdb.com//title/tt0072684/,(1975),1976-09-10


In [7]:
df['datetime'] = pd.to_datetime(df['date'], format = '%d %B %Y')

df['month'] = df.datetime.dt.month
df['month'] = df.datetime.dt.day
df['month'] = df.datetime.dt.year

In [8]:
def preparedatetime(dataframe):
    dataframe['datetime'] = pd.to_datetime(dataframe['date'], format = '%d %B %Y')
    dataframe['month'] = dataframe.datetime.dt.month
    dataframe['month'] = dataframe.datetime.dt.day
    dataframe['month'] = dataframe.datetime.dt.year

preparedatetime(df)

In [9]:
def find_budget(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Budget:":
            print(i.text.strip())
            return float(i.text.strip().split("\n")[0].split("$")[1].replace(",", ""))
        
find_budget(s)

NameError: name 's' is not defined

In [ ]:
def find_runtime(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Runtime:":
            return i.text.strip().split("\n")[1].split("min")[0].strip()
find_runtime(soup_)

In [ ]:
url_ = links[0]
r = requests.get(url_)

url_

In [ ]:
soup_ = BeautifulSoup(r.text,'html.parser')

In [ ]:
def find_genre(soup):
    for i in soup.find(id="title-overview-widget").find_all("div", class_="subtext"):
        return str(i.text.strip().split("|")[2].replace('\n','').replace(' ','').split(","))
        #return i.text.strip().split("|")[2].replace('\n','').replace(' ','').split(",")

find_genre(soup_)



In [ ]:
def find_genre(soup):
    for i in soup.find(id="title-overview-widget").find_all("div", class_="subtext"):
        if str(i.text.strip().split("|")[2].replace('\n','').replace(' ','').split(","))[-3]==(')'):
            return i.text.strip().split("|")[1].replace('\n','').replace(' ','').split(",")
        else:
            return i.text.strip().split("|")[2].replace('\n','').replace(' ','').split(",")

find_genre(soup_)


In [ ]:
rating = soup_.find_all('div', class_="ratingValue")

rating_array=[]
for r in rating:
    try:
        rating_1=r.find("span", class_="ratingValue")
        rating_array.append(rating_1.getText())
    except:
        rating_array.append(0)

    
rating_array



In [ ]:
for genre in yo:
        genre
        return genre